In [1]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Boolean, Date, ForeignKey, Sequence, CheckConstraint, UniqueConstraint

In [9]:
%run WWTBAM_import.py

FileNotFoundError: [Errno 2] No such file or directory: 'WWTBAM.json'

In [2]:
Base = declarative_base()

C:\Users\domin\AppData\Local\Temp\ipykernel_18356\4196137762.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
# create tables structure
class AppContent(Base):
    __tablename__ = 'AppContent'
    __table_args__ = (
        UniqueConstraint('tag'),
    )
    id = Column(Integer, primary_key=True)
    tag = Column(String(50), nullable=False)
    original_language_id = Column(Integer, ForeignKey('Languages.id'))
    original_text = Column(String(500))

    def __repr__(self):
        return "<AppContent(id={0}, tag={1}, original_language_id={2}, original_text={3})>".format(
            self.id, self.tag, self.original_language_id, self.original_text)
       
class AppTranslations(Base):
    __tablename__ = 'AppTranslations'
    id = Column(Integer, primary_key=True)
    text_content_id = Column(Integer, ForeignKey("AppContent.id"))
    language_id = Column(Integer, ForeignKey("Languages.id"))
    translation = Column(String(500))

    def __repr__(self):
        return "<AppTranslations(id={0}, text_content_id={1}, language_id={2}, translation={3})>".format(
            self.id, self.text_content_id, self.language_id, self.translation)
    
class Languages(Base):
    __tablename__ = 'Languages'
    __table_args__ = (
        CheckConstraint('LENGTH(name) = 2'),
        UniqueConstraint('name'),
    )
    id = Column(Integer, primary_key=True)
    name = Column(String(50), nullable = False)

    def __repr__(self):
        return "<Languages(id={0}, name={1})>".format(
            self.id, self.name)
    
class Questions(Base):
    __tablename__ = 'Questions'
    id = Column(Integer, primary_key=True)
    original_language_id = Column(Integer, ForeignKey("Languages.id"))
    question = Column(String(500))
    correct_answer = Column(String(50))
    wrong_answer_1 = Column(String(50))
    wrong_answer_2 = Column(String(50))
    wrong_answer_3 = Column(String(50))
    used = Column(Boolean, unique=False, default=False)

    def __repr__(self):
        return "<Questions(id={0}, original_language_id={1}, question={2}, correct_answer={3}, wrong_answer_1={4}, wrong_answer_2={5}, wrong_answer_3={6}, used={7})>".format(
            self.id, self.original_language_id, self.question, self.correct_answer, self.wrong_answer_1, self.wrong_answer_2, self.wrong_answer_3, self.used)
    
class QuestionTranslations(Base):
    __tablename__ = 'QuestionTranslations'
    id = Column(Integer, primary_key=True)
    question_id = Column(Integer, ForeignKey("Questions.id"))
    language_id = Column(Integer, ForeignKey("Languages.id"))
    question = Column(String(500))
    correct_answer = Column(String(50))
    wrong_answer_1 = Column(String(50))
    wrong_answer_2 = Column(String(50))
    wrong_answer_3 = Column(String(50))

    def __repr__(self):
        return "<QuestionTranslations(id={0}, question_id={1}, language_id={2}, question={3}, correct_answer={4}, wrong_answer_1={5}, wrong_answer_2={6}, wrong_answer_2={7})>".format(
            self.id, self.question_id, self.language_id, self.question, self.correct_answer, self.wrong_answer_1, self.wrong_answer_2, self.wrong_answer_3)

In [4]:
# connect to database
engine = create_engine("postgresql://postgres@localhost/millionaires")

# create database if not exists
if not database_exists(engine.url):
    create_database(engine.url)
    Base.metadata.create_all(engine)

db = engine.connect()

In [5]:
from sqlalchemy import MetaData, Table, insert

# create tables using ORM techniques
metadata = MetaData()
languages = Table('Languages', metadata, autoload_with=db)
text_content = Table('AppContent', metadata, autoload_with=db)

# insert original language
languages_query = insert(languages).values(name='pl')

# insert original text content
text_content_query = insert(text_content).values(
[
    {'tag': '50_50', 'original_text': '50 / 50', 'original_language_id': 1},
    {'tag': 'call_friend', 'original_text': 'Telefon do przyjaciela', 'original_language_id': 1},
    {'tag': 'question_public', 'original_text': 'Pytanie do publiczności', 'original_language_id': 1},
    {'tag': 'question', 'original_text': 'Pytanie', 'original_language_id': 1},
    {'tag': 'play_further', 'original_text': 'Czy chcesz grać dalej?', 'original_language_id': 1},
    {'tag': 'lifebuoy', 'original_text': 'Koła ratunkowe', 'original_language_id': 1},
    {'tag': 'question_value', 'original_text': 'Wartość pytania', 'original_language_id': 1},
    {'tag': 'game_result', 'original_text': 'Wynik gry', 'original_language_id': 1},
    {'tag': 'millionaires', 'original_text': 'Milionerzy', 'original_language_id': 1},
    {'tag': 'correct_answer', 'original_text': 'Brawo, to jest poprawna odpowiedź! :)', 'original_language_id': 1},
    {'tag': 'million_win', 'original_text': "Wygrałeś Milion !!!\nJeśli chcesz zagrać jeszcze raz kliknij przycisk 'Rozpocznij grę'", 'original_language_id': 1},
    {'tag': 'incorrect_answer', 'original_text': "Niestety, jest to zła odpowiedź! :(\nPoprawną odpowiedzią była odpowiedź ", 'original_language_id': 1},
    {'tag': 'no_win_incorrect_answer', 'original_text': "Niestety nic dziś nie wygrałeś :(\nJeśli chcesz zagrać jeszcze raz kliknij przycisk 'Rozpocznij grę'", 'original_language_id': 1},
    {'tag': '1000_win_incorrect_answer', 'original_text': "Wygrałeś 1000 zł\nJeśli chcesz zagrać jeszcze raz kliknij przycisk 'Rozpocznij grę'", 'original_language_id': 1},
    {'tag': '40000_win_incorrect_answer', 'original_text': "Wygrałeś 40000 zł\nJeśli chcesz zagrać jeszcze raz kliknij przycisk 'Rozpocznij grę'", 'original_language_id': 1},
    {'tag': 'question_for', 'original_text': 'Pytanie za ', 'original_language_id': 1},
    {'tag': 'no_win_resignation', 'original_text': 'Niestety nic dziś nie wygrałeś/łaś :(', 'original_language_id': 1},
    {'tag': 'x_win_resignation', 'original_text': 'Wygrałeś/łaś ', 'original_language_id': 1},
    {'tag': 'currency', 'original_text': 'zł', 'original_language_id': 1},
    {'tag': 'correct_answer_resignation', 'original_text': 'Poprawną odpowiedzią była odpowiedź ', 'original_language_id': 1},
    {'tag': 'lifebuoy_used', 'original_text': 'To koło ratunkowe zostało już użyte', 'original_language_id': 1},
    {'tag': 'friend_hint', 'original_text': 'Wydaje mi się, że jest to odpowiedź ', 'original_language_id': 1},
    {'tag': 'start_game', 'original_text': 'Rozpocznij grę', 'original_language_id': 1},
    {'tag': 'resignation', 'original_text': 'Rezygnuję', 'original_language_id': 1},
    {'tag': 'rules', 'original_text': 'Zasady gry Milionerzy', 'original_language_id': 1},
    {'tag': 'go_to_game', 'original_text': 'Przejdź do gry', 'original_language_id': 1}  
]
)
# execute queries
result_languages = db.execute(languages_query)
result_text_content = db.execute(text_content_query)
db.commit()